In [36]:
# 步骤 2：导入必要的库
import os
from openai import OpenAI
import numpy as np
import faiss
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import logging
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

# 配置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [37]:
# 步骤 4：加载嵌入模型

# 加载嵌入模型
device = torch.device("cpu")  # 强制使用CPU
try:
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
    model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2').to(device)
    logging.info("嵌入模型加载成功！")
except Exception as e:
    logging.error(f"加载嵌入模型时发生错误: {e}")


2024-11-30 18:56:21,851 - INFO - 嵌入模型加载成功！


In [38]:
# 步骤 5：定义嵌入函数

# 嵌入函数（批量处理）
def embed_texts(texts, batch_size=16):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors='pt', padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        # 使用CLS token的输出作为句子的嵌入
        batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)


In [39]:
# 步骤 6：加载文档数据

# 加载文档数据（CSV文件）
def load_documents(file_path, nrows=100):
    try:
        df = pd.read_csv(file_path, nrows=nrows)
        df = df.dropna(subset=['text'])
        logging.info(f"成功加载了 {len(df)} 条文档。")
        return df
    except FileNotFoundError:
        logging.error(f"文件 {file_path} 未找到。请检查路径是否正确。")
        return pd.DataFrame(columns=['text'])
    except Exception as e:
        logging.error(f"加载文档时发生错误: {e}")
        return pd.DataFrame(columns=['text'])


In [40]:
# 步骤 7：构建FAISS索引

# 构建FAISS索引
def build_faiss_index(embeddings, use_quantization=False):
    dimension = embeddings.shape[1]
    
    if use_quantization:
        # 使用Product Quantization进行压缩
        nlist = 100  # 聚类数
        quantizer = faiss.IndexFlatL2(dimension)
        index = faiss.IndexIVFPQ(quantizer, dimension, nlist, 16, 8)  # 16 bytes per vector, 8 subquantizers
        index.train(embeddings)
        index.add(embeddings)
        logging.info("使用量化的FAISS索引已构建。")
    else:
        # 使用简单的扁平索引（不量化）
        index = faiss.IndexFlatL2(dimension)
        index.add(embeddings)
        logging.info("使用扁平FAISS索引已构建。")
    
    return index


In [41]:
# 步骤 8：检索相关文档

# 检索相关文档
def retrieve_relevant_documents(index, query_embedding, texts, top_k=2):
    distances, indices = index.search(np.array([query_embedding]), top_k)
    return [texts[i] for i in indices[0]]


In [42]:
# 步骤 9：生成回答

# 设置API密钥和基础URL
llm = OpenAI(
    api_key="sk-16a90ba86cfc4dcf9402bea1309c9021",
    base_url="https://api.deepseek.com"
)

# 使用DeepSeek生成回答
def generate_response(prompt):
    response = llm.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    return response.choices[0].message.content.strip()


In [43]:
# 步骤 10：主流程

# 主流程
def main(use_csv=True, file_path="documents.csv", nrows=100, use_quantization=False, top_k=2):
    try:
        if use_csv:
            # 使用CSV加载文档
            logging.info("正在加载CSV文档...")
            df = load_documents(file_path, nrows=nrows)
            
            if df.empty:
                logging.error("未加载到任何文档。请检查CSV文件。")
                return
            
            texts = df['text'].tolist()
        else:
            # 使用默认文档列表
            logging.info("未使用CSV，使用默认文档。")
            texts = [
                "这是第一段默认的文本。",
                "这是第二段默认的文本。",
                "这是第三段默认的文本。"
            ]
            
            if not texts:
                logging.error("默认文档列表为空。")
                return
        
        # 生成嵌入
        logging.info("正在生成嵌入...")
        embeddings = embed_texts(texts)
        
        # 构建FAISS索引
        logging.info("正在构建FAISS索引...")
        faiss_index = build_faiss_index(embeddings, use_quantization=use_quantization)
        
        # 获取用户输入（这里使用固定的问题）
        user_input = "CHIMA是谁?"
        
        # 生成查询嵌入
        logging.info("正在生成查询嵌入...")
        query_embedding = embed_texts([user_input])[0]
        
        # 检索相关文档
        logging.info("正在检索相关文档...")
        relevant_docs = retrieve_relevant_documents(faiss_index, query_embedding, texts, top_k=top_k)
        
        if not relevant_docs:
            logging.warning("未检索到相关文档。")
            return
        
        # 将检索到的文档作为上下文
        context = "\n".join(relevant_docs)
        logging.info(f"检索到的上下文内容如下：\n{context}")
        
        # 使用DeepSeek生成回答
        logging.info("正在生成AI回答...")
        #prompt = f"根据以下上下文回答用户问题：\n\n上下文：\n{context}\n\n问题：\n{user_input}"
        prompt = f"根据以下上下文回答用户问题：\n\n上下文：\n{context}\n\n问题：\n{user_input}。如果缺少上下文则根据你的知识回答。"
        ai_response = generate_response(prompt)
        
        print("\nAI回答：")
        print(ai_response)
    
    except Exception as e:
        logging.error(f"发生错误: {e}")


In [44]:
# 步骤 11：运行主流程

# 运行主流程，使用CSV加载文档
main(use_csv=True, file_path="documents.csv", nrows=100, use_quantization=False, top_k=2)
#main(use_csv=False)


2024-11-30 18:56:25,454 - INFO - 正在加载CSV文档...
2024-11-30 18:56:25,542 - INFO - 成功加载了 3 条文档。
2024-11-30 18:56:25,548 - INFO - 正在生成嵌入...
2024-11-30 18:56:26,053 - INFO - 正在构建FAISS索引...
2024-11-30 18:56:26,058 - INFO - 使用扁平FAISS索引已构建。
2024-11-30 18:56:26,059 - INFO - 正在生成查询嵌入...
2024-11-30 18:56:26,081 - INFO - 正在检索相关文档...
2024-11-30 18:56:26,085 - INFO - 检索到的上下文内容如下：
中国医院协会信息专业委员会(CHIMA)，为中国医院协会所属的分支机构，是从事医院信息化的非营利性、群众性的行业学术组织。CHIMA的主要工作：开展国内外医院信息学术交流活动，制定有关医院信息标准规范及规章制度，培训和提高医院信息人员专业水平，从而推动中国医院信息化工作事业的发展。CHIMA的前身是中华医院管理学会的计算机应用学组，于1985年成立，由时任北京民航总医院院长的王志明教授和北京协和医院计算机室主任的李包罗教授任学组组长和副组长。1997年，中华医院管理学会独立成为一级学会，1998年8月，成立了中华医院管理学会信息管理专业委员会(CHIMA)，由李包罗任主任委员。2006年2月，中华医院管理学会改名为“中国医院协会”，学会亦改名为中国医院协会信息管理专业委员会。2019年换届后更名为“中国医院协会信息专业委员会”，由原国家卫生健康委统计信息中心副主任王才有担任主任委员。经过40年发展，CHIMA已经发展成为中国最大的医疗卫生信息化专业学术团体，形成覆盖医疗、卫生各个领域HIT基础和应用研究的专业组织体系，是中国HIT界最具影响力和领导力的学术组织之一。截至2023年6月换届，CHIMA委员共计422位，其中主任委员1位、副主任委员13位、秘书长1位、常务委员119位。
北京卫生信息技术协会(PHITA)，是由北京地区从事卫生信息技术和管理的个人、医疗机构以及信息技术企业自愿组成的、实行行业服务和自律管理的非营利性社会团体。协会宗旨


AI回答：
CHIMA是中国医院协会信息专业委员会（China Hospital Information Management Association）的简称。它是中国医院协会所属的分支机构，是一个从事医院信息化的非营利性、群众性的行业学术组织。CHIMA的主要工作包括开展国内外医院信息学术交流活动，制定有关医院信息标准规范及规章制度，培训和提高医院信息人员专业水平，从而推动中国医院信息化工作事业的发展。CHIMA的前身是中华医院管理学会的计算机应用学组，成立于1985年，经过多年的发展，已经成为中国最大的医疗卫生信息化专业学术团体之一。
